In [1]:
%cd ..

/home/zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.utils.vis_utils import model_to_dot
from keras_tqdm import TQDMNotebookCallback
from keras import backend as K
from tqdm import tqdm_notebook

from data import zero_filled_2d_generator
from unet import unet

Using TensorFlow backend.


In [3]:
# paths
train_path = '/media/zaccharie/BACKUP/singlecoil_train/'
val_path = '/home/zaccharie/workspace/fastmri-reproducible-benchmark/singlecoil_val'
test_path = '/home/zaccharie/workspace/fastmri-reproducible-benchmark/singlecoil_test'

In [4]:
n_samples_train = 34742
n_samples_val = 7135

In [5]:
# generators
batch_size = 2
train_gen = zero_filled_2d_generator(train_path, batch_size=batch_size)
val_gen = zero_filled_2d_generator(val_path, batch_size=batch_size)

In [6]:
run_params = {'n_layers': 4, 'pool': 'average', "layers_n_channels": [1, 2, 4, 8]}
n_epochs = 2
run_id = 'second_test'
chkpt_path = f'checkpoints/{run_id}'

In [7]:
chkpt_cback = ModelCheckpoint(chkpt_path)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    batch_size=batch_size, 
    write_graph=True, 
    write_images=True, 
)

In [8]:
model = unet(input_size=(320, 320, 1), with_extra_sigmoid=True, **run_params)

print(model.summary())
model.fit_generator(
    train_gen, 
    steps_per_epoch=int(n_samples_train / batch_size), 
    epochs=n_epochs,
    validation_data=val_gen,
    validation_steps=int(n_samples_val / batch_size),
    verbose=0,
    callbacks=[TQDMNotebookCallback(), tboard_cback, chkpt_cback],
#     max_queue_size=100,
#     use_multiprocessing=True,
#     workers=3,
)

W0715 18:38:36.151227 140675430696768 deprecation_wrapper.py:119] From /home/zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 18:38:36.153033 140675430696768 deprecation_wrapper.py:119] From /home/zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0715 18:38:36.157414 140675430696768 deprecation_wrapper.py:119] From /home/zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0715 18:38:36.174357 140675430696768 deprecation_wrapper.py:119] From /home/zaccharie/workspace/fastmri-reproducible-be

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 320, 320, 1)  10          input_1[0][0]                    
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 160, 160, 1)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 160, 160, 2)  20          average_pooling2d_1[0][0]        
__________________________________________________________________________________________________
average_po

W0715 18:38:38.411201 140675430696768 deprecation_wrapper.py:119] From /home/zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0715 18:38:38.412221 140675430696768 deprecation_wrapper.py:119] From /home/zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



KeyError: "Unable to open object (object 'mask' doesn't exist)"

In [ ]:
# K.clear_session() 